In [ ]:
!pip install shap

     |████████████████████████████████| 564 kB 14.4 MB/s 


In [ ]:
from sklearn.preprocessing import QuantileTransformer
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn import linear_model
import xgboost as xgb

In [ ]:
!gdown 14AK65LfYoj7aSjZOXB9HUzbhqaBaxQi0

Downloading...
From: https://drive.google.com/uc?id=14AK65LfYoj7aSjZOXB9HUzbhqaBaxQi0
To: /content/Matrix_complexity.csv
100% 1.39M/1.39M [00:00<00:00, 103MB/s]


In [ ]:
data_all = pd.read_csv('/content/Matrix_complexity.csv')

In [ ]:
models = ['XGBoost']
metrics = ['MAPE', 'MAE', 'R2']
formes = ['inverted', 'average']
columns = [
    np.array([models[i] for i in range(1) for j in range(3) for k in range(2)]),
    np.array([metrics[j] for i in range(1) for j in range(3) for k in range(2)]),
    np.array([formes[k] for i in range(1) for j in range(3) for k in range(2)]),
]
chans = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']


In [ ]:
import tqdm

result = pd.DataFrame()
grid_res = []
n_films = 8
n_subjects = 32
for column in data_all.columns:
    data_all[column] = data_all[column].fillna(0)


for film in tqdm.tqdm_notebook(range(1, n_films + 1)):

  df = pd.DataFrame(np.random.randn(1, 1 * 3 * 2), columns=columns)
  X_test = data_all.loc[(data_all['film'] == film)].reset_index(drop=True)
  X_train = data_all.loc[(data_all['film'] != film)].reset_index(drop=True)
  X_train = X_train.sample(frac=1)
  X_test = X_test.sample(frac=1)

  y_train = X_train['Labels']
  y_test_inv = np.array(X_test['Labels'])
  train_film = X_train['film']
  train_subj = X_train['Subj']
  test_film = X_test['film']
  test_subj = X_test['Subj']
  
  X_train = X_train.drop(['film', 'Subj', 'Labels'], axis=1)
  X_test = X_test.drop(['film', 'Subj', 'Labels'], axis=1)
  
  qt_train = QuantileTransformer(output_distribution='normal',
                            copy=True)
  y_train = pd.DataFrame(qt_train.fit_transform(np.array(y_train).reshape(-1, 1)))
  qt_test = QuantileTransformer(output_distribution='normal',
                            copy=True)
  y_test = pd.DataFrame(qt_test.fit_transform(np.array(y_test_inv).reshape(-1, 1)))

  model = xgb.XGBClassifier()
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)
  preds = qt_test.inverse_transform(np.array(y_pred).reshape(-1, 1))
  df[('XGBoost', 'R2', 'inverted')] = r2_score(preds, y_test_inv)
  df[('XGBoost', 'MAPE', 'inverted')] = mean_absolute_percentage_error(preds, y_test_inv)
  df[('XGBoost', 'MAE', 'inverted')] = mean_absolute_error(preds, y_test_inv)
  
  pred = [[0 for j in range(n_films)] for i in range(n_subjects)]
  num_chans = [[0 for j in range(n_films)] for i in range(n_subjects)]
  labels = [[0 for j in range(n_films)] for i in range(n_subjects)]
  lbl = [[0 for j in range(n_films)] for i in range(n_subjects)]
  channels = [[] for i in range(n_subjects)]
  for i in range(len(preds)):
      pred[np.array(test_subj)[i] - 1][np.array(test_film)[i] - 1] += preds[i]
      num_chans[np.array(test_subj)[i] - 1][np.array(test_film)[i] - 1] += 1
      labels[np.array(test_subj)[i] - 1][np.array(test_film)[i] - 1] += y_test_inv[i]
  labels = np.array(labels).astype(np.float64) / np.array(num_chans).astype(np.float64)
  pred = np.array(pred).astype(np.float64) / np.array(num_chans).astype(np.float64)
  labels = np.nan_to_num(labels)
  pred = np.nan_to_num(pred)
  predicted = []
  lbl = []
  for i in range(len(labels)):
      if sum(labels[i]) != 0:
          for j in range(n_films):
              if labels[i][j] != 0:
                  lbl.append(labels[i][j])  
                  predicted.append(pred[i][j])

  df[('XGBoost', 'R2', 'average')] = r2_score(lbl, predicted)
  df[('XGBoost', 'MAPE', 'average')] = mean_absolute_percentage_error(lbl, predicted)
  df[('XGBoost', 'MAE', 'average')] = mean_absolute_error(lbl, predicted)
  result = pd.concat([result, df], axis=0)

In [ ]:
result.describe()

XGBoost                                                  
           MAPE                 MAE                  R2          
       inverted   average  inverted   average  inverted   average
count  8.000000  8.000000  8.000000  8.000000  8.000000  8.000000
mean   0.444383  0.467913  2.091037  2.091037 -1.098300 -0.375083
std    0.217431  0.175910  0.666490  0.666490  1.019821  0.404492
min    0.246500  0.263866  1.289655  1.289655 -3.130085 -1.015566
25%    0.281878  0.370628  1.611553  1.611553 -1.588601 -0.622356
50%    0.384486  0.412777  2.096230  2.096230 -0.878030 -0.427020
75%    0.523030  0.570201  2.295304  2.295304 -0.419393 -0.066432
max    0.887408  0.753999  3.396825  3.396825  0.149224  0.247978